In [ ]:
#目標:
# 將 easycamp_reviews_cleaned.json 中的滿意度資料，
# 透過比對 campground_merge.營地名稱，取得對應的 營地ID，
# 更新 campground 資料表中同一個 ID 的評分欄位。

In [1]:
import os
os.getcwd()

'c:\\TJR101_03_Project\\connect'

In [1]:
#將專案的根目錄加入到 sys.path
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


In [2]:
import pandas as pd
import json
import pymysql
#引入將json轉df的函式
from transform.campground_table_rating import create_rating_df

name               object
traffic_rating      Int64
bathroom_rating     Int64
view_rating         Int64
service_rating      Int64
facility_rating     Int64
dtype: object


In [3]:
rating_df = create_rating_df()

name               object
traffic_rating      Int64
bathroom_rating     Int64
view_rating         Int64
service_rating      Int64
facility_rating     Int64
dtype: object


In [4]:
rating_df.dtypes

name               object
traffic_rating      Int64
bathroom_rating     Int64
view_rating         Int64
service_rating      Int64
facility_rating     Int64
dtype: object

In [5]:
import pymysql
import datetime

In [ ]:
# 連線到 MySQL 
conn = pymysql.connect(
        host="localhost", 
        user="root", 
        password="your_password", 
        database="easycamp_db"
    )
    
cursor = conn.cursor()

In [7]:
for _, row in rating_df.iterrows():
    name = row['name']
     # 轉換 <NA> 成 None，MySQL 才能識別
    ratings = tuple(
        val if pd.notna(val) else None
        for val in (
            row['traffic_rating'],
            row['bathroom_rating'],
            row['view_rating'],
            row['service_rating'],
            row['facility_rating']
        )
    )
    # 找到對應的 campground_id
    cursor.execute("SELECT campground_id FROM campground_merge WHERE camping_site_name = %s", (name,))
    result = cursor.fetchone()

    if result:
        campground_id = result[0]

        # 更新 campground 表格
        cursor.execute("""
            UPDATE campground
            SET traffic_rating = %s,
                bathroom_rating = %s,
                view_rating = %s,
                service_rating = %s,
                facility_rating = %s
            WHERE campground_id = %s
        """, ratings + (campground_id,))
    else:
        print(f"❗找不到營地：{name}")

❗找不到營地：牧匠露營區


In [8]:
# 提交變更
conn.commit()
cursor.close()
conn.close()